# Swift BAT Light Curve Analysis with Bayesian Blocks

This notebook demonstrates how to use `grb_pipeline` to read Swift BAT catalog light curves,
compute optimal Bayesian Block representations, and create multi-band stacked plots.

The data are downloaded from the [Swift GRB catalog](https://swift.gsfc.nasa.gov/results/batgrbcat/).

Authors: Vikas Chand, Partha S. Pal, Dimple

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from grb_pipeline.analysis.lightcurve import (
    LightCurveAnalyzer, LightCurveData, BATLightCurve,
    read_bat_catalog_lc, plot_bat_multiband
)

## 1. Read BAT Catalog Light Curve

BAT light curves from the catalog come as ASCII files with 9 columns:
time, band1_rate, band1_err, band2_rate, band2_err, band3_rate, band3_err, band4_rate, band4_err

Energy bands: 15-25 keV, 25-50 keV, 50-100 keV, 100-350 keV

In [ ]:
# Read a BAT catalog light curve file
# Replace with your actual data file
GRB_name = 'GRB181126A'
binsize_ms = 64

filename = f"{GRB_name}_{binsize_ms}ms_lc_ascii.dat.txt"
bands = read_bat_catalog_lc(filename)

print(f"Loaded {GRB_name} with {len(bands)} energy bands")
for name, lc in bands.items():
    print(f"  {name}: {len(lc.time)} bins, rate range [{lc.rate.min():.4f}, {lc.rate.max():.4f}]")

## 2. Compute Bayesian Blocks

We use `astropy.stats.bayesian_blocks` with the 'measures' fitness function
and a false-alarm probability p0=0.01 to find the optimal piecewise-constant
representation of each light curve band.

In [ ]:
# Using the BATLightCurve class for an integrated workflow
bat_lc = BATLightCurve.from_catalog_file(filename)

# Run Bayesian blocks on all bands
bat_lc.run_bayesian_blocks(p0=0.01)

for name, result in bat_lc.bayesian_blocks_results.items():
    print(f"  {name}: {len(result['edges'])-1} blocks")

## 3. Multi-band Stacked Plot

The four BAT energy bands are plotted in a vertically stacked arrangement.
Raw data is shown with transparency, overlaid with the Bayesian Block representation
in black for each band.

In [ ]:
# Method 1: Using the BATLightCurve class
fig = bat_lc.plot(xlim=(-2.5, 10))
plt.show()

In [ ]:
# Method 2: Using the standalone function directly
fig = plot_bat_multiband(bands, grb_name=GRB_name, xlim=(-2.5, 10), p0=0.01)
plt.show()

## 4. Using LightCurveAnalyzer for Individual Band Analysis

You can also use the general `LightCurveAnalyzer` for more detailed analysis
of individual bands, such as T90 calculation and peak flux measurement.

In [ ]:
analyzer = LightCurveAnalyzer()

# Get the broadband (15-350 keV) light curve
broadband = bat_lc.get_broadband()
print(f"Broadband light curve: {len(broadband.time)} bins")

# Compute Bayesian blocks on broadband
bb_result = analyzer.bayesian_blocks(broadband, p0=0.01)
print(f"Broadband Bayesian blocks: {len(bb_result['edges'])-1} blocks")
print(f"Block edges: {bb_result['edges'][:5]}... (showing first 5)")

In [ ]:
# Calculate T90 on the broadband light curve
t90_result = analyzer.calculate_t90(broadband)
print(f"T90 = {t90_result['t90']:.2f} s")
print(f"T90 start = {t90_result['t_start']:.2f} s")
print(f"T90 stop = {t90_result['t_stop']:.2f} s")

# Peak flux
pf = analyzer.peak_flux(broadband)
print(f"\nPeak flux = {pf['peak_flux']:.4f} counts/s/det")
print(f"Peak time = {pf['peak_time']:.3f} s")